# Lab 5: Thermal data processing

**Purpose:** The purpose of this lab is to become familiar with thermal data sets and practice processing time series information with Earth Engine.


In [ ]:
%pylab inline

In [ ]:
# import ee api and geemap package
import ee
import math
import geemap
import pandas as pd
from geemap import colormaps as cmaps

In [ ]:
# try to initalize an ee session
# if not authenticated then run auth workflow and initialize
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

## Land Surface Temperature from MODIS

Land surface temperature can either be extracted from derived products, such as the MODIS Terra and Aqua satellite products (Wan 2006), or estimated directly from measurements in the thermal band. We will only explore using the precomputed data using the city of Atlanta, GA, USA, as the region of interest.


In [ ]:
# read in a vector file for the cities of GA
cities = ee.FeatureCollection("users/kelmarkert/public/cities_georgia")
# filter for the boundaries 
atl = ee.Feature(cities.filter(ee.Filter.eq("Name","Atlanta")).first())

Note: The city boundary data was downloaded from the [Atlanta Regional Commission Open data hub](https://opendata.atlantaregional.com/datasets/34520575dfc34b8cac783caff702b8cc_58/explore) and uploaded to EE.

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.centerObject(atl, 11)

Map.addLayer(cities, {"color":"yellow"}, "GA Cities")
Map.addLayer(atl, {}, 'Atlanta');

Map.addLayerControl()

Map

Next we load in the MODIS MYD11A2 version 6 product, which provides 8-day composites of LST from the Aqua satellite. This corresponds to an equatorial crossing time of roughly 1:30 p.m. during daytime and 1:30 a.m. at night. In contrast, the MODIS sensor onboard the Terra platform (MOD11A2 version 6) has an overpass of ~10:30 a.m. and ~10:30 p.m. 

In [ ]:
# Load MODIS image collection from the Earth Engine data catalog.
modis_lst = ee.ImageCollection("MODIS/006/MYD11A2")

To make the data usable we need to convert the LST to degrees Celsius and mask out all poor quality pixels (which include cloud and water observations). We will do so by first creating a function to extract out QA mask then scale to degrees Celsius and then map the function over the image colletion.

In [ ]:
def extract_bits(image, start, end=None, new_name=None):
    """Function to convert qa bits to binary flag image

    args:
        image (ee.Image): qa image to extract bit from
        start (int): starting bit for flag
        end (int | None, optional): ending bit for flag, if None then will only use start bit. default = None
        new_name (str | None, optional): output name of resulting image, if None name will be {start}Bits. default = None

    returns:
        ee.Image: image with extract bits
    """

    newname = new_name if new_name is not None else f"{start}_bits"

    if (start == end) or (end is None):
        # perform a bit shift with bitwiseAnd
        return image.select([0], [newname]).bitwiseAnd(1 << start)
    else:
        # Compute the bits we need to extract.
        pattern = 0
        for i in range(start, end):
            pattern += int(math.pow(2, i))

        # Return a single band image of the extracted QA bits, giving the band
        # a new name.
        return image.select([0], [newname]).bitwiseAnd(pattern).rightShift(start)

def preprocess(image):
    qa_band = image.select("QC_Day")

    mask = extract_bits(qa_band, start=2, end=3).eq(0)

    return image.multiply(0.02).subtract(273.15).updateMask(mask).copyProperties(image,["system:time_start"])


This function is more complex as the bits required for

In [ ]:
# apply the preprocessing function and select the day band
lst_c = modis_lst.map(preprocess).select("LST_Day_1km")

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.centerObject(atl, 11)

Map.addLayer(lst_c.mean(), {"min":15,"max":35,"palette":cmaps.get_palette("inferno")}, "LST")
Map.addLayer(atl, {}, 'Atlanta');

Map.addLayerControl()

Map

In [ ]:
task = ee.batch.Export.table.toAsset(
    collection = ee.FeatureCollection(ref),
    description = "CE594 Example Export",
    assetId = "your/asset/name" #"users/kmarkert/ce594/example_table_export"
)

In [ ]:
task.start()

In [ ]:
image_task = ee.batch.Export.image.toAsset(
    image = lst_c.mean(),
    region = atl.buffer(1000).geometry(),
    scale=1000,
    assetId = "your/asset/name" #"users/kmarkert/ce594/example_timage_export"
)

image_task.start()

### LST Time series

We have a handle on the processing for LST and now we would like to calculate the temperature in time for Atlanta. To do so, we will need to map a function over the collection to find the average temerature.

In [ ]:
# define a function to calculate the avg. temperature for ATL
def atl_temp(image):
    # reduction function
    temp = image.reduceRegion(
        reducer = ee.Reducer.mean(),
        geometry = atl.geometry(),
        scale = 1000
    )

    # set the result as a metadata property in the image
    return image.set(temp)

# apply the function and filter for images that were not all masked
lst_c_atl = lst_c.map(atl_temp).filter(ee.Filter.neq("LST_Day_1km",None))

Note: there are other methods to calculate a time series such as `getRegion()`, however, processing things using the `ImageCollection` data structure is best for processing.

In [ ]:
# extract out the timeseries information from the collection
timeseries = lst_c_atl.aggregate_array("LST_Day_1km").getInfo()
timestamp = lst_c_atl.aggregate_array("system:time_start").getInfo()

In [ ]:
# convert the data into a pandas DataFrame
dates = pd.to_datetime(np.array(timestamp)*1e6)
atl_series = pd.Series(timeseries,index=dates,name="ATL LST")

In [ ]:
atl_series.plot(figsize=(10,7));

Next we want to compare the land surface temperature to a reference area. We will use the interactive map get the geometry...

In [ ]:
# get the drawn features
ref = ee.FeatureCollection(Map.draw_features)

In [ ]:
ref

In [ ]:
Map.addLayer(ref,{},"Reference geom")

In [ ]:
# define a function to calculate the avg. temperature for the reference geom
def ref_temp(image):
    # reduction function
    temp = image.reduceRegion(
        reducer = ee.Reducer.mean(),
        geometry = ref.geometry(),
        scale = 1000
    )

    # set the result as a metadata property in the image
    return image.set(temp)

# apply the function and filter for images that were not all masked
lst_c_ref = lst_c.map(ref_temp).filter(ee.Filter.neq("LST_Day_1km",None))

In [ ]:
# extract out the timeseries information from the collection
timeseries = lst_c_ref.aggregate_array("LST_Day_1km").getInfo()
timestamp = lst_c_ref.aggregate_array("system:time_start").getInfo()

In [ ]:
# convert the data into a pandas DataFrame
dates = pd.to_datetime(np.array(timestamp)*1e6)
ref_series = pd.Series(timeseries,index=dates,name="Ref LST")

In [ ]:
# combine the ATL and reference series
df = pd.concat([atl_series, ref_series], axis=1)

In [ ]:
df.plot(figsize=(10,7));

### More filtering

We will use this as an opportunity to explore more filtering functions. For example, if we are interested in the average for only a specific period within a year we can do so. 

Here we will create an average spring and autumn image of LST and compare the two results.

In [ ]:
# define a filter to only spring months
spring_filter = ee.Filter.calendarRange(3,5,"month")

# apply spring filter on lst dataset
spring_lst = lst_c.filter(spring_filter)

In [ ]:
autumn_filter = ee.Filter.calendarRange(9,11,"month")

# apply autumn filter on lst dataset
autumn_lst = lst_c.filter(autumn_filter)

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.centerObject(atl, 11)

Map.addLayer(spring_lst.mean(), {"min":15,"max":35,"palette":cmaps.get_palette("inferno")}, "Spring LST")
Map.addLayer(autumn_lst.mean(), {"min":15,"max":35,"palette":cmaps.get_palette("inferno")}, "Autumn LST")

Map.addLayer(atl, {}, 'Atlanta');

Map.addLayerControl()

Map

Another common use case would be to calculate monthly average values. This requires creating a list of months to calculate, mapping over each one, and filter/reduce for that month

In [ ]:
# define a function to filter by month and average
def monthly_mean(i):
    # cast the value as a number
    i = ee.Number(i)
    # filter by the month and 
    return lst_c.filter(ee.Filter.calendarRange(i, field="month")).mean()

# list of values to map over
months = ee.List.sequence(1,12)

# apply function and cast to an image collection
monthly_lst = ee.ImageCollection.fromImages(months.map(monthly_mean))

In [ ]:
# create a gif of the monthly LST
url = monthly_lst.getVideoThumbURL({
    "min":5,
    "max":40,
    "palette":cmaps.get_palette("inferno"),
    "region":atl.geometry().buffer(8e5).bounds(),
    "framesPerSecond":3,
    "format":"gif",
    "dimensions":1000
})

url